**1. Upload the video file to data folder**

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Відкриваємо файли
cap = cv2.VideoCapture("data/Match.mp4")
template = cv2.imread("data/template.png", 0)
h, w = template.shape[:2]

In [3]:
output_size = (1080, 1920)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('data/output2.mp4', fourcc, 30.0, output_size)

In [4]:
# 4 опорні точки з кадру з відео)
pts_src = np.array([[210, 60], [945, 60], [1113, 1240], [114, 1240]], dtype=np.float32)

# відповідні точки на "випрямленому" зображенні
pts_dst = np.array([[0, 0], [1080, 0], [1080, 1920], [0, 1920]], dtype=np.float32)

# Матриця гомографії для вирівнювання перспективи
H, _ = cv2.findHomography(pts_src, pts_dst)

In [5]:
# Параметри для Lucas-Kanade Optical Flow
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

tracking = None
prev_gray = None

⚠️ **To Fix:** Трекінг в цілому працює, але коли гравець різко б'є по м'ячу, то трекінг переключається на гравця і застрягає на ньому

In [6]:
threshold = 0.83  # для шаблонного пошуку
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # перетворення перспективи до кадру
    warped = cv2.warpPerspective(frame, H, (1080, 1920))
    gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY) # у сірий
    
    if tracking is None:
        # якщо трекінг не активний — шукаємо м'яч через шаблон
        res = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        print(f"accuracy: {max_val:.3f}")

        # якщо результат співпадіння перевищує трешхолд — об'єкт знайдено
        if max_val >= threshold:
            top_left = max_loc
            
            bottom_right = (top_left[0] + w, top_left[1] + h)
            center_x = top_left[0] + w // 2
            center_y = top_left[1] + h // 2
            tracking = np.array([[center_x, center_y]], dtype=np.float32)
            
            cv2.rectangle(warped, top_left, bottom_right, (255, 0, 0), 2)
            cv2.putText(warped, f'Detected {max_val:.3f}', (top_left[0], top_left[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    else:
        # Трекінг за допомогою Optical Flow
        next_point, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray, tracking, None, **lk_params)
        
        if status[0][0] == 1:
            x, y = next_point[0]
            tracking = next_point
            cv2.circle(warped, (int(x), int(y)), 10, (0, 0, 255), 2)
            cv2.putText(warped, "Ball (tracked)", (int(x) + 10, int(y)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        else:
            tracking = None  # Втрата трекінгу

    # Записуємо оброблений кадр у відео
    out.write(warped)
    prev_gray = gray.copy()
    
    # зменшую розмір кадру в 2 рази для зручного перегляду
    scaled_warped = cv2.resize(warped, (0, 0), fx=1, fy=1)
    cv2.imshow("Warped View", scaled_warped)

    # вихід по клавіші Esc
    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

accuracy: 0.768
accuracy: 0.770
accuracy: 0.774
accuracy: 0.782
accuracy: 0.761
accuracy: 0.763
accuracy: 0.771
accuracy: 0.769
accuracy: 0.780
accuracy: 0.773
accuracy: 0.790
accuracy: 0.772
accuracy: 0.783
accuracy: 0.791
accuracy: 0.785
accuracy: 0.777
accuracy: 0.773
accuracy: 0.782
accuracy: 0.807
accuracy: 0.796
accuracy: 0.784
accuracy: 0.774
accuracy: 0.776
accuracy: 0.777
accuracy: 0.775
accuracy: 0.772
accuracy: 0.776
accuracy: 0.758
accuracy: 0.774
accuracy: 0.783
accuracy: 0.777
accuracy: 0.783
accuracy: 0.780
accuracy: 0.770
accuracy: 0.798
accuracy: 0.790
accuracy: 0.785
accuracy: 0.777
accuracy: 0.783
accuracy: 0.785
accuracy: 0.765
accuracy: 0.773
accuracy: 0.776
accuracy: 0.774
accuracy: 0.790
accuracy: 0.793
accuracy: 0.796
accuracy: 0.774
accuracy: 0.774
accuracy: 0.765
accuracy: 0.762
accuracy: 0.800
accuracy: 0.790
accuracy: 0.774
accuracy: 0.778
accuracy: 0.779
accuracy: 0.772
accuracy: 0.785
accuracy: 0.780
accuracy: 0.775
accuracy: 0.777
accuracy: 0.798
accuracy